In [54]:
#https://habr.com/ru/company/ods/blog/328372/



from sklearn.metrics import log_loss
from sklearn.model_selection import cross_val_score

from time import time

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
%matplotlib inline

import matplotlib.pyplot as plt  # Matlab-style plotting
import seaborn as sns
color = sns.color_palette()
sns.set_style('darkgrid')
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)

from scipy import stats
from scipy.stats import norm, skew #for some statistics

pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) #Limiting floats output to 3 decimal points

from subprocess import check_output


#for log-loss 
labels=[0, 1]

In [55]:
from time import time

tik = time()
tik_local = time()

def mytime(tik_local):
    tak = time()
    print('all time is - ', tak-tik, '(sec)')
    print('local time is - ', tak-tik_local, '(sec)')
    return tak

tik_local = mytime(tik_local)

all time is -  0.0002353191375732422 (sec)
local time is -  0.00018477439880371094 (sec)


In [56]:
# подсчет строк в файле train.tar.gz
# 29 989 753
#CPU times: user 5min 51s, sys: 9.14 s, total: 6min 1s
#Wall time: 6min 5s

# подсчет строк в файле test.tar.gz
# 1 317 220


import tarfile as tf
import gzip as gz
#from StringIO import StringIO

tik = time()

infile = 'mailru_data/materials/train.tar.gz'
def linecount(infile, member):
    lc = 0
    with gz.GzipFile(infile) as zipf:
        with tf.TarFile(fileobj=zipf) as tarf:
            dataf = tarf.extractfile(member)
            while dataf.readline():
               lc += 1 
            dataf.close()       
    return lc
#print(linecount(infile, 'train.csv'))
tik_local = mytime(tik_local)

all time is -  0.00025963783264160156 (sec)
local time is -  0.012349605560302734 (sec)


In [57]:
#determinate train to numeric and categoric data
# Take analysis of numeric and categorical columns
def columns_len(all_data):
    numeric_feats = all_data.dtypes[all_data.dtypes != "object"].index
    start_len_num = len(numeric_feats)
    print('len of the numeric_feats is: ', start_len_num)

    categorical_feats = all_data.dtypes[all_data.dtypes == "object"].index
    start_len_cat = len(categorical_feats)
    print('len of the categorical_feats is: ', start_len_cat)
    print('total len of all_data_feats is: ', len(all_data.columns))
    return numeric_feats, categorical_feats

In [58]:
#Unique Data Analysis
def unique_analysis(all_data, unique_part):
    col_for_drop = []
    for col in all_data.columns:
        if (np.max(all_data[col].value_counts())/len(all_data[col]))>unique_part:
            print(col,np.max(all_data[col].value_counts()), ' of ', len(all_data[col]), ' per cent is ', 
                  np.max(all_data[col].value_counts())*100//len(all_data[col]),'%')
            col_for_drop.append(col)
    print
    return col_for_drop

#aa = unique_analysis(0.8)
#print(aa)

In [59]:
def list_columns(all_data):
    cat_cols = [c for c in all_data.columns if all_data[c].dtype.name == 'object']
    num_cols   = [c for c in all_data.columns if all_data[c].dtype.name != 'object']
    return num_cols, cat_cols

#print (columns_len.numeric_feats)

**Загрузка данных**
-------------------

In [60]:
%%time
#Wall time: 1min 51s
nrows = 5*10**3
#nrows = 29989753   #29 989 753

train = pd.read_csv('mailru_data/materials/train.tar.gz', compression='gzip', header=0, sep=';', quotechar='"', nrows=nrows)
train = train[:-1]
#train = train.drop([29989752])

print('------ train shape -----')
print(train.columns[0])
print(len(train.columns))
print(train.shape)




#Analyst of test dataframe
nrows_test = 4*10**3
#nrows_test= 1317220  #1 317 220

test = pd.read_csv('mailru_data/materials/test-data.tar.gz', compression='gzip', header=0, sep=',', quotechar='"', nrows=nrows_test)
test.drop("Unnamed: 0", axis = 1, inplace = True)

print('------ test shape -----')
print(test.columns[0])
print(test.shape)
print('-----------------------')

tik_local = mytime(tik_local)


------ train shape -----
train.csv
19
(4999, 19)
------ test shape -----
test.csv
(4000, 19)
-----------------------
all time is -  0.0896143913269043 (sec)
local time is -  0.0893547534942627 (sec)
CPU times: user 51.1 ms, sys: 196 µs, total: 51.3 ms
Wall time: 52.1 ms


**datasets ans targets**
-------------------------------------------

In [61]:
#Отделение столбца таргета и соединения тренировочной и тестовой части для общей обработки данных

train.rename_axis('Id')
test.rename_axis('Id')

#print(train[:5])
train_ID = train['train.csv']
test_ID = test['test.csv']

train.drop('train.csv', axis = 1, inplace = True)
test.drop("test.csv", axis = 1, inplace = True)

ntrain = train.shape[0]
ntest = test.shape[0]
y_train = train['label'].values
y_test = test['label'].values
all_data = pd.concat((train, test)).reset_index(drop=True)
all_data.drop(['label'], axis=1, inplace=True)
print('train len - ',ntrain, 'test len - ',ntest)
print("all_data size is : {}".format(all_data.shape))
#print(all_data[:3])

tik_local = mytime(tik_local)

train len -  4999 test len -  4000
all_data size is : (8999, 17)
all time is -  0.11810302734375 (sec)
local time is -  0.028488636016845703 (sec)


**Train & test analisyst**
--------------------------

In [62]:
train.dtypes

label     int64
C1        int64
C2        int64
C3        int64
C4        int64
C5        int64
C6        int64
C7        int64
C8        int64
C9        int64
C10       int64
CG1      object
CG2      object
CG3      object
l1        int64
l2        int64
C11       int64
C12       int64
dtype: object

In [63]:
train.head()


,label,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,CG1,CG2,CG3,l1,l2,C11,C12
0,0,2733540231,3500392421,4454,15573,11,995,2,176,15,671,"384,382,96,88,185,49,385,268,448,438,279,420,1...",NaN,NaN,32,0,0,106
1,0,1964843810,1232327635,2081,2547,11,995,2,361,15,802,"96,49,385,268,107,438,418,124,357,17,435,130,3...",NaN,NaN,1,0,0,106
2,0,660357674,2066571765,602,29581,29,995,0,468,15,758,"170,169,205,204,176,252,243,382,98,220,222,181...",NaN,NaN,119,0,0,103
3,0,2962648192,2373358995,4018,36969,30,995,2,8,15,67,"273,205,383,382,381,219,216,215,225,49,47,54,3...","28071,6916,9108",NaN,0,0,0,106
4,0,1822613420,446973819,2081,23457,35,995,0,452,15,821,"268,419,357,18,331,59,57,99,154,155,76,412,122...",NaN,NaN,5,0,0,106


In [64]:
test['label']=test['label']*(-1)
test.head()

,label,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,CG1,CG2,CG3,l1,l2,C11,C12
0,1,4210358866,3196051971,3336,2094,22,995,0,361,15,566,"176,213,220,222,224,47,422,103,419,74,3,235,23...",NaN,NaN,0,0,0,103
1,1,2248128334,3994996972,393,16232,52,995,2,8,15,821,"379,384,98,96,215,224,186,188,183,184,49,50,47...","17179,3883,20334,20333,25423,5318,10989,8668,2...",NaN,6,0,0,104
2,1,2323433900,1100964395,4454,34157,11,995,2,361,15,672,"243,380,269,420,363,31,18,214,331,60,58,306,19...",NaN,NaN,48,0,0,106
3,1,3430094722,1690426382,5919,6384,25,2658,0,390,22,722,"448,277,276,275,99,154,155,76,412,139,333,332,...","22337,2002","43074,43841,45509,45500,33701,54846,38344,1159...",2,0,1,106
4,1,3531271529,3678868632,1169,26182,30,995,0,94,15,1131,"273,170,205,204,174,173,248,384,382,381,92,97,...",16129,NaN,23,0,0,104


In [65]:
def draw_histograms(dataframe, features, rows, cols):
    fig=plt.figure(figsize=(20,20))
    for i, feature in enumerate(features):
        ax=fig.add_subplot(rows,cols,i+1)
        dataframe[feature].hist(bins=20,ax=ax,facecolor='midnightblue')
        ax.set_title(feature+" Distribution",color='DarkRed')
        
    fig.tight_layout()  
    plt.show()

#draw_histograms(all_data[:ntrain],all_data.columns,6,3)
tik_local = mytime(tik_local)

all time is -  0.180833101272583 (sec)
local time is -  0.06273007392883301 (sec)


In [66]:
print(train['label'].unique())
print(test['label'].unique())
print(test['label'].sum())

[0 1]
[1]
4000


In [67]:
# Проверка уникальностей для каждого столбца train
for col in all_data.columns:
    if len(all_data[col][:ntrain].unique())<11: 
        print(col, len(all_data[col][:ntrain].unique()),all_data[col][:ntrain].unique() , ' ------- ', col, len(all_data[col][ntrain:].unique()),all_data[col][ntrain:].unique() )
    else: print(col, len(all_data[col][:ntrain].unique()), ' ------- ', col, len(all_data[col][ntrain:].unique()))
        
tik_local = mytime(tik_local)

C1 2372  -------  C1 1961
C2 1799  -------  C2 1399
C3 783  -------  C3 649
C4 1363  -------  C4 1091
C5 63  -------  C5 64
C6 281  -------  C6 225
C7 3 [2 0 1]  -------  C7 3 [0 2 1]
C8 109  -------  C8 96
C9 46  -------  C9 44
C10 173  -------  C10 159
CG1 2306  -------  CG1 1916
CG2 624  -------  CG2 486
CG3 369  -------  CG3 242
l1 207  -------  l1 222
l2 5 [0 1 2 4 5]  -------  l2 8 [ 0  1  3  2 21  6  9  5]
C11 2 [0 1]  -------  C11 2 [0 1]
C12 8 [106 103 105 104 107 100 101 102]  -------  C12 7 [103 104 106 105 100 107 102]
all time is -  0.2457873821258545 (sec)
local time is -  0.06495428085327148 (sec)


**Feature ingeniring**
----------------------

In [68]:
num_cols, cat_cols = list_columns(all_data)

for col in num_cols:
    for coli in num_cols:
        if col != coli:
            str1 = col + '_plus_' + coli
            str2 = col + '_minus_' + coli
            all_data[str1] = all_data[col] + all_data[coli]
            all_data[str2] = all_data[col] - all_data[coli]
            #print(col, '---', coli)


for col in all_data.select_dtypes('object'):
    all_data[col] = all_data[col].astype(str)

print(all_data.shape)
tik_local = mytime(tik_local)

(8999, 381)
all time is -  0.6453709602355957 (sec)
local time is -  0.3995835781097412 (sec)


In [69]:
all_data_str = all_data[['C5','C7', 'C9', 'l2', 'C11', 'C12']]
all_data_str.columns = ['C5_str','C7_str', 'C9_str', 'l2_str', 'C11_str', 'C12_str']

for col in all_data.select_dtypes('object'):
    all_data[col] = all_data[col].astype(str)
    
for col in all_data_str.columns:
    all_data_str[col] = all_data_str[col].astype(str)

all_data = pd.concat([all_data, all_data_str], axis=1)

print(all_data.shape)
num_cols, cat_cols = list_columns(all_data)

(8999, 387)


**Outliers**
-------------

In [71]:
# Определение вылетов по межквартильному размаху в столбцах с цифрами
# Помечаем вылеты через дополнительные столбцы col +'Outlier'

#создаем один признак

#feature = all_data[num_cols[0]]
#создаем функцию, которая возвращает индекс выбросов
def indicies_of_outliers(x):
    q1, q3 = np.percentile(x, [25, 75])
    iqr = q3-q1
    lower_bond = q1 - (iqr*1.5)
    upper_bond = q3 + (iqr*1.5)
    return np.where((x > upper_bond) | (x < lower_bond))

#применяем функцию
# делаем дополнительные бинарные столбы с признаком выброса по каждому числовому столбцу

for col in num_cols:   #all_data.columns:
    pp = indicies_of_outliers(all_data[col])
    list_index = pp[0]
    if len(list_index)>1:
        name_col = col +'Outlier'
        all_data[name_col] = 0
        all_data[name_col][list_index] = 1
        print(len(list_index), ' --- ', all_data[name_col].sum(), ' --- ', name_col)
         
columns_len(all_data)
#print(all_data['TotRmsOutlier'][:15])
tik_local = mytime(tik_local)

1135  ---  1135  ---  C5Outlier
1203  ---  1203  ---  C6Outlier
408  ---  408  ---  C8Outlier
1246  ---  1246  ---  C9Outlier
1474  ---  1474  ---  l1Outlier
87  ---  87  ---  l2Outlier
1203  ---  1203  ---  C11Outlier
187  ---  187  ---  C12Outlier
len of the numeric_feats is:  386
len of the categorical_feats is:  9
total len of all_data_feats is:  395
all time is -  171.19701981544495 (sec)
local time is -  170.55164885520935 (sec)


**Correlations**
---------------

In [73]:
# Correlation matrix
def corr_data(features, target):
    #обработка высококоррелированных признаков
    #создаем корреляционную матрицу
    print('количество признаков вначале:', features.shape[1])
    corr_matrix = features.corr().abs()

    #выбираем верхний треугольник корреляционной матрицы (т.к. нижний симметричный)
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

    #находим индекс столбцов признаком с корреляцией больше 0.95
    to_drop = [column for column in upper.columns if any(upper[column]>0.95)]
    print(to_drop)

    # исключаем эти признаки
    features.drop(to_drop, axis =1).head(3)

    print('количество признаков:', features.shape[1])
    

corr_data(all_data[:ntrain], y_train)
tik_local = mytime(tik_local)

количество признаков вначале: 395
['C1_plus_C3', 'C1_minus_C3', 'C1_plus_C4', 'C1_minus_C4', 'C1_plus_C5', 'C1_minus_C5', 'C1_plus_C6', 'C1_minus_C6', 'C1_plus_C7', 'C1_minus_C7', 'C1_plus_C8', 'C1_minus_C8', 'C1_plus_C9', 'C1_minus_C9', 'C1_plus_C10', 'C1_minus_C10', 'C1_plus_l1', 'C1_minus_l1', 'C1_plus_l2', 'C1_minus_l2', 'C1_plus_C11', 'C1_minus_C11', 'C1_plus_C12', 'C1_minus_C12', 'C2_plus_C1', 'C2_minus_C1', 'C2_plus_C3', 'C2_minus_C3', 'C2_plus_C4', 'C2_minus_C4', 'C2_plus_C5', 'C2_minus_C5', 'C2_plus_C6', 'C2_minus_C6', 'C2_plus_C7', 'C2_minus_C7', 'C2_plus_C8', 'C2_minus_C8', 'C2_plus_C9', 'C2_minus_C9', 'C2_plus_C10', 'C2_minus_C10', 'C2_plus_l1', 'C2_minus_l1', 'C2_plus_l2', 'C2_minus_l2', 'C2_plus_C11', 'C2_minus_C11', 'C2_plus_C12', 'C2_minus_C12', 'C3_plus_C1', 'C3_minus_C1', 'C3_plus_C2', 'C3_minus_C2', 'C3_plus_C4', 'C3_minus_C4', 'C3_plus_C5', 'C3_minus_C5', 'C3_plus_C7', 'C3_minus_C7', 'C3_plus_C8', 'C3_minus_C8', 'C3_plus_C9', 'C3_minus_C9', 'C3_plus_C10', 'C3_minus_

**Отбор признаков, ранжирование по важности**
-----------------

In [75]:
#рекурсивное устранение признаков после нормирования признаков

from sklearn.feature_selection import RFECV
from sklearn.linear_model import LinearRegression
import warnings


warnings.filterwarnings(action='ignore', module = 'scipy', message='^internal gelsd')

features = all_data[:ntrain][num_cols]
target = y_train

ols = LinearRegression()
rfecv = RFECV(estimator=ols, step=1, scoring='neg_mean_squared_error')
rfecv.fit(features, target)
rfecv.transform(features)

print(rfecv.n_features_, 'количество наилучших признаков')
print("какие категории самые лучшие: \n", rfecv.support_)
print("ранжирование признака от самого лучшего (1) до самого плохого: \n", rfecv.ranking_)


ii_col = pd.concat([pd.DataFrame(all_data.columns, columns=['feature']), pd.DataFrame(rfecv.ranking_, columns=['ranking'])], axis=1)
ii_col = ii_col.sort_values(by=['ranking'])

best_features = ii_col[ii_col['ranking']<15]['feature']
#print(ii_col)
print(best_features)
tik_local = mytime(tik_local)

2 количество наилучших признаков
какие категории самые лучшие: 
 [False False False False False False False False False False False  True
  True False]
ранжирование признака от самого лучшего (1) до самого плохого: 
 [13 12 11 10  4  9  2  5  6  7  8  1  1  3]
11    CG2
12    CG3
6      C7
13     l1
4      C5
7      C8
8      C9
9     C10
10    CG1
5      C6
3      C4
2      C3
1      C2
0      C1
Name: feature, dtype: object
all time is -  296.7330951690674 (sec)
local time is -  84.56405091285706 (sec)


**preprocessing pipeline**
---------------------

In [88]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression, LogisticRegression

X_train = all_data[:ntrain]
X_test = all_data[ntrain:]

num_cols, cat_cols = list_columns(all_data)
    
numeric_features = num_cols
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_features = cat_cols
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

clf = Pipeline(steps=[('preprocessor', preprocessor),
                  ('regressor', LogisticRegression())])  

#clf.fit(X_train,y_train)

def get_feature_names(self):
        return self.X.columns.tolist()
    
#get_feature_names(clf)

#print(X_train.columns.tolist())
clf.fit(X_train, y_train)
#y_pred = clf.predict(X_train)
print(clf.score(X_train, y_train))

print(clf.score(X_test, y_test))

clf.fit_transform(X_train, y_train)

#X_train = preprocessor.fit_transform(X_train)
#X_train

0.9975995199039808
0.00025


AttributeError: 'LogisticRegression' object has no attribute 'transform'

In [89]:
num_cols, cat_cols = list_columns(all_data)
print(cat_cols)

['CG1', 'CG2', 'CG3', 'C5_str', 'C7_str', 'C9_str', 'l2_str', 'C11_str', 'C12_str']


In [16]:
# Заполнение Nan

all_data["CG1"] = all_data["CG1"].fillna(0)
all_data["CG2"] = all_data["CG2"].fillna(0)
all_data["CG3"] = all_data["CG3"].fillna(0)

all_data['C7'] = all_data['C7'].fillna(train['C7'].mode()[0])
all_data['l2'] = all_data['l2'].fillna(train['l2'].mode()[0])
all_data['C11'] = all_data['C11'].fillna(train['C11'].mode()[0])
all_data['C12'] = all_data['C12'].fillna(train['C12'].mode()[0])


all_data['CG1'] = all_data['CG1'].apply(lambda x: str(x).count(',')+1) 
all_data['CG2'] = all_data['CG2'].apply(lambda x: str(x).count(',')+1) 
all_data['CG3'] = all_data['CG3'].apply(lambda x: str(x).count(',')+1)
all_data['CG1'] = all_data['CG1'].apply(lambda x: float(x))
all_data['CG2'] = all_data['CG2'].apply(lambda x: float(x))
all_data['CG3'] = all_data['CG3'].apply(lambda x: float(x))
all_data['CCG3'] = all_data['CG3']+all_data['CG1']
all_data['CCG4'] = all_data['CG3']+all_data['CG2']
all_data['CCG5'] = all_data['CG1']+all_data['CG2']
all_data['C71'] = all_data['C3']*all_data['C4']
all_data['C72'] = all_data['C3']*all_data['C8']
all_data['C73'] = all_data['C3']*all_data['C5']
all_data['C74'] = all_data['C3']*all_data['C6']
all_data['C75'] = all_data['C3']*all_data['C7']
all_data['C76'] = all_data['C3']*all_data['C9']
all_data['C77'] = all_data['C4']*all_data['C5']
all_data['C78'] = all_data['C4']*all_data['C6']
all_data['C79'] = all_data['C4']*all_data['C7']


print(all_data[["CG1", 'CG2', 'CG3']][:5])

columns_len(all_data)
num_cols, cat_cols = list_columns(all_data)
#print(num_cols)

tik_local = mytime(tik_local)

     CG1   CG2   CG3
0 40.000 1.000 1.000
1 30.000 1.000 1.000
2 54.000 1.000 1.000
3 46.000 3.000 1.000
4 19.000 1.000 1.000
len of the numeric_feats is:  29
len of the categorical_feats is:  0
total len of all_data_feats is:  29
all time is -  1.0445668697357178 (sec)
local time is -  0.08758139610290527 (sec)


In [17]:
# генерирование полиноминальных признаков
def polinominal():
    from sklearn.preprocessing import PolynomialFeatures

    polynominal_interaction = PolynomialFeatures(degree=2, include_bias=False)

    polynominal_interaction.fit_transform(all_data)

    columns_len(all_data)
    num_cols, cat_cols = list_columns(all_data)
    all_data.shape

In [18]:
# Detect Outliers - определение выбросов через элипс
def ellipce_outliers():
    from sklearn.covariance import EllipticEnvelope

    #make detector
    outlier_detector = EllipticEnvelope(contamination = .1)
    #fit of detector
    outlier_detector.fit(all_data[num_cols])
    #predict of outliers
    p = outlier_detector.predict(all_data[num_cols])
    print(p)
    print(len(p))
    pq = p-1
    nn = pq.sum()
    print(nn)

all time is -  1.240380048751831 (sec)
local time is -  0.009525775909423828 (sec)


In [78]:
#Skewed features

# Check the skew of all numerical features
skewed_feats = all_data.apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
print("\nSkew in numerical features: \n")
skewness = pd.DataFrame({'Skew' :skewed_feats})
skewness.head(10)

#Box Cox Transformation of (highly) skewed features

skewness = skewness[abs(skewness) > 0.75]    # My_DS_work - was 0.75 my vers - 0.5
print("There are {} skewed numerical features to Box Cox transform".format(skewness.shape[0]))

from scipy.special import boxcox1p
skewed_features = skewness.index
lam = 0.15
for feat in skewed_features:
    #all_data[feat] += 1
    all_data[feat] = boxcox1p(all_data[feat], lam)
    
#all_data[skewed_features] = np.log1p(all_data[skewed_features])

#Getting dummy categorical features

all_data = pd.get_dummies(all_data)
print(all_data.shape)
all_data.head()
tik_local = mytime(tik_local)

TypeError: ("unsupported operand type(s) for /: 'str' and 'int'", 'occurred at index CG1')

In [79]:
#рекурсивное устранение признаков после нормирования признаков через бокс-кокс

from sklearn.feature_selection import RFECV
from sklearn.linear_model import LinearRegression
import warnings


warnings.filterwarnings(action='ignore', module = 'scipy', message='^internal gelsd')

features = all_data[:ntrain]
target = y_train

ols = LinearRegression()
rfecv = RFECV(estimator=ols, step=1, scoring='neg_mean_squared_error')
rfecv.fit(features, target)
rfecv.transform(features)

print(rfecv.n_features_, 'количество наилучших признаков')
print("какие категории самые лучшие: \n", rfecv.support_)
print("ранжирование признака от самого лучшего (1) до самого плохого: \n", rfecv.ranking_)


ii_col = pd.concat([pd.DataFrame(all_data.columns, columns=['feature']), pd.DataFrame(rfecv.ranking_, columns=['ranking'])], axis=1)
ii_col = ii_col.sort_values(by=['ranking'])

best_features = ii_col[ii_col['ranking']<15]['feature']
#print(ii_col)
print(best_features)
tik_local = mytime(tik_local)

ValueError: could not convert string to float: '384,382,96,88,185,49,385,268,448,438,279,420,124,123,5,3,17,133,71,409,330,59,57,302,304,395,275,151,113,99,154,155,76,412,139,333,332,335,334,399'

In [80]:
from keras import models
from keras import layers

network = models.Sequential()
network.add(layers.Dense(units=16, activation='relu', input_shape = (len(all_data.columns),)))
network.add(layers.Dense(units=16, activation='relu'))
network.add(layers.Dense(units=1, activation='sigmoid'))
network.compile(
    loss='binary_crossentropy',
    optimizer = 'rmsprop',
    metrics=['accuracy'])

history = network.fit(
    all_data[:ntrain],
    y_train,
    epochs=3,
    verbose=1,
    batch_size = 100,
    validation_data = (all_data[ntrain:], y_test))

predict_target = network.predict(all_data[ntrain:])



tik_local = mytime(tik_local)

Using TensorFlow backend.
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarn

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 4999 samples, validate on 4000 samples
Epoch 1/3


ValueError: could not convert string to float: '205,380,384,382,98,96,220,222,186,188,183,49,47,53,54,52,385,336,337,171,100,279,280,161,74,123,208,15,239,372,341,133,71,409,10,214,202,306,194,322,99,154,155,76,412,122,139,333,332,335,334,137,399'

In [81]:
predict_target = predict_target[:,0]
#y_test= y_test.apply(lambda x: float(x))
#print(y_test[:2].values)
#print(predict_target[:2])
print('log_loss neuron network', log_loss(y_test, predict_target, labels=labels))
tik_local = mytime(tik_local)

NameError: name 'predict_target' is not defined

In [82]:
from sklearn.metrics import log_loss
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.model_selection import StratifiedKFold

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier


log_loss_scorer=make_scorer(log_loss, needs_proba=True)
rf=RandomForestClassifier(n_estimators=100, random_state=2017, n_jobs=-1)
scores=cross_val_score(rf, X=all_data[:ntrain], y=y_train, cv=3, scoring=log_loss_scorer, verbose=1)
print(scores, 'RandomForestClassifier')

#разделение на 2 класса через опорно-векторную машину на ядерных функциях
from sklearn.svm import SVC

svc = SVC(kernel='rbf', class_weight = 'balanced', random_state=0, C=1, probability=True)
scores=cross_val_score(rf, X=all_data[:ntrain], y=y_train, cv=3, scoring=log_loss_scorer, verbose=1)
print(scores, 'SVC')
tik_local = mytime(tik_local)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


ValueError: could not convert string to float: '96,215,52,385,268,416,449,104,279,280,161,74,124,123,357,235,31,18,202,57,308,152,351,150,113,324,323,99,154,155,76,412,122,200,329,139,333,332,335,334,399'

In [26]:
from sklearn.ensemble import ExtraTreesClassifier

clf = ExtraTreesClassifier(
        n_estimators = 50,
        max_depth = None,
        max_features = 'auto',
        criterion='entropy',
        n_jobs = -1,
        random_state = 0)
        
    
scores=cross_val_score(rf, X=all_data[:ntrain], y=y_train, cv=3, scoring=log_loss_scorer, verbose=1)
print(scores, 'ExtraTreesClassifier')

tik_local = mytime(tik_local)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[0.08552275 0.06807468 0.08586559] ExtraTreesClassifier
all time is -  10.288185358047485 (sec)
local time is -  0.9901361465454102 (sec)


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    1.0s finished


In [27]:
#(page 231)отбор наилучших моделей из нескольких обучающихся алгоритмов

def find_best_algor():
    import numpy as np
    from sklearn import datasets
    from sklearn.linear_model import LogisticRegression
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import GridSearchCV
    from sklearn.pipeline import Pipeline

    #задать случайное число для генератора псевдослучайных чисел
    np.random.seed(0)

    features = all_data[best_features][:ntrain]
    target = y_train


    #создать конвейер
    pipe = Pipeline([('classifier', RandomForestClassifier())])
    #создать словарь вариантов обучающихся алгоритмов и их гиперпараметров
    search_space = [{'classifier': [LogisticRegression()],
                    'classifier__penalty': ['l1', 'l2'],
                    'classifier__C': np.logspace(0, 4, 10)},
                    {'classifier': [RandomForestClassifier()],
                     'classifier__n_estimators': [10, 100, 1000],
                     'classifier__max_features': [1, 2, 3]}]
    #создать объект решеточного поиска
    gridsearch = GridSearchCV(pipe, search_space, cv=5, verbose=0)
    best_model = gridsearch.fit(features, target)
    #взглянуть на лучшую модель и ее параметры
    best_model.best_estimator_.get_params()['classifier']

In [28]:
#(page 238)оценивание результативности после отбора модели
def val_scormodel():
    from sklearn import linear_model
    from sklearn.model_selection import GridSearchCV, cross_val_score

    logistic = linear_model.LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                       intercept_scaling=1, l1_ratio=None, max_iter=100,
                       multi_class='warn', n_jobs=-1, penalty='l1',
                       random_state=None, solver='warn', tol=0.0001, verbose=0,
                       warm_start=False)

    #создать диапазон из 20 вариантов значений для С
    C = np.logspace(0, 4, 20)
    #создать словарь вариантов гиперпараметров
    hyperparameters = dict(C=C)
    #создать объект решеточного поиска
    gridsearch = GridSearchCV(logistic, hyperparameters, cv=5, n_jobs=-1, verbose=0)
    #выполнить вложенную перекрестную проверку и выдать среднюю оценку
    cross_val_score(gridsearch, features, target).mean()



In [29]:
import numpy as np
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

def any_any():
    features = all_data[best_features][:ntrain]
    target = y_train
    test_pred = all_data[best_features][ntrain:]

    #создать объект предобработки, который включает
    #признаки стандартного шкалировщика и объект РСА
    preprocess = FeatureUnion([('std', StandardScaler()), ('pca', PCA())])
    #создать конвейер
    pipe = Pipeline([('preprocess', preprocess),
                    ('classifier', LogisticRegression())])
    #создать пространство вариантов значений
    search_space = [{'preprocess__pca__n_components': [1, 2, 3],
                     'classifier__penalty': ['l1', 'l2'],
                     'classifier__C': np.logspace(0, 4, 10)}]

    #создать объект решеточного поиска
    clf = GridSearchCV(pipe, search_space, cv=5, verbose=0, n_jobs=-1)
    best_model = clf.fit(features, target)
    #взглянуть на лучшую модель и ее параметры
    best_model.best_estimator_.get_params()['preprocess__pca__n_components']

In [30]:
from sklearn.metrics.classification import accuracy_score, log_loss
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF

train_size = ntrain
X = all_data
y_test = test['label'].values
print(y_train)
print(y_test)
y = pd.concat([pd.DataFrame(y_train), pd.DataFrame(y_test)], ignore_index=True) #.reset_index(drop=True))

print(X[:train_size].shape, y[:train_size].shape)
print(X[train_size:].shape, y[train_size:].shape)


[0 0 0 ... 0 0 0]
[1 1 1 ... 1 1 1]
(4999, 50) (4999, 1)
(4000, 50) (4000, 1)


In [31]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

logistic = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=-1, penalty='l1',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

logistic.fit(X[:train_size], y[:train_size])
log_pred = logistic.predict_log_proba(X[train_size:])
#log_pred1 = logistic.predict(X[train_size:])
#print(log_pred)
#print(log_pred1)
#print("Log-loss: %.3f (logistic)" % log_loss(y[train_size:], log_pred1))
print(log_loss(y_test, log_pred, labels=labels))
tik_local = mytime(tik_local)

0.6931471805599454
all time is -  13.377265930175781 (sec)
local time is -  3.089080572128296 (sec)


In [32]:
log_pred1 = pd.DataFrame(log_pred)
log_pred1 = log_pred1[log_pred1.columns[0]]
#print(log_pred)
#print(log_pred1)


log_pred1.to_csv('sample_submission.csv', header=0, index=False)
#log_pred1.to_csv('sample_submission3.csv',index=False)

print('finish')
#print('nrows_test', nrows_test)
#print('log_pred', len(log_pred), 'shape', log_pred.shape)
#print('log_pred1', len(log_pred1), 'shape', log_pred1.shape)


finish


In [33]:
from sklearn.metrics.classification import accuracy_score, log_loss
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF

train_size = ntrain
X = all_data
y_test = test['label'].values
print(y_train)
print(y_test)
y = pd.concat([pd.DataFrame(y_train), pd.DataFrame(y_test)], ignore_index=True) #.reset_index(drop=True))

print(X[:train_size].shape, y[:train_size].shape)
print(X[train_size:].shape, y[train_size:].shape)


# Specify Gaussian Processes with fixed and optimized hyperparameters
gp_fix = GaussianProcessClassifier(kernel=1.0 * RBF(length_scale=1.0),
                                   optimizer=None)
gp_fix.fit(X[:train_size], y[:train_size])

gp_opt = GaussianProcessClassifier(kernel=1.0 * RBF(length_scale=1.0))
gp_opt.fit(X[:train_size], y[:train_size])

print("Log Marginal Likelihood (initial): %.3f"
      % gp_fix.log_marginal_likelihood(gp_fix.kernel_.theta))
print("Log Marginal Likelihood (optimized): %.3f"
      % gp_opt.log_marginal_likelihood(gp_opt.kernel_.theta))

print("Accuracy: %.3f (initial) %.3f (optimized)"
      % (accuracy_score(y[:train_size], gp_fix.predict(X[:train_size])),
         accuracy_score(y[:train_size], gp_opt.predict(X[:train_size]))))
print("Log-loss: %.3f (initial) %.3f (optimized)"
      % (log_loss(y[:train_size], gp_fix.predict_proba(X[:train_size], labels=labels)[:, 1]),
         log_loss(y[:train_size], gp_opt.predict_proba(X[:train_size], labels=labels)[:, 1])))


tik_local = mytime(tik_local)

[0 0 0 ... 0 0 0]
[1 1 1 ... 1 1 1]
(4999, 50) (4999, 1)
(4000, 50) (4000, 1)


KeyboardInterrupt: 

In [ ]:
fix_pred = gp_fix.predict_proba(X[train_size:])
opt_pred = gp_opt.predict_proba(X[train_size:])
print(fix_pred)
print(opt_pred)

fix_pred = pd.DataFrame(fix_pred)
opt_pred = pd.DataFrame(opt_pred)
fix_pred.to_csv('sample_submission2.csv',index=False)  #'sample_submission.csv'
opt_pred.to_csv('sample_submission3.csv',index=False)